<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/05_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-25 15:31:45--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12099 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.82K  --.-KB/s    in 0s      

2022-06-25 15:31:45 (84.1 MB/s) - ‘utils.py’ saved [12099/12099]



In [1]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 16
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [7]:
target_shape = (32, 48, 1)

rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL, trainable=True)


rgb_features = rgb_hub_module(rgb_input)
rgb_layer = keras.layers.Reshape(target_shape=target_shape)(rgb_features)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)
xyz_layer = keras.layers.Reshape(target_shape=target_shape)(xyz_features)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)
hsv_layer = keras.layers.Reshape(target_shape=target_shape)(hsv_features)

block = keras.layers.Concatenate()([rgb_layer, xyz_layer, hsv_layer])

conv1 = keras.layers.Conv2D(filters=10,
                            kernel_size=(2,2),
                            strides=(2,3),
                            padding='same',
                            activation='relu')(block)

conv2 = keras.layers.Conv2D(filters=20,
                            kernel_size=(2,2),
                            strides=(2,2),
                            activation='relu')(conv1)

avg_pool = keras.layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = keras.layers.Conv2D(filters=20,
                            kernel_size=(1,1),
                            activation='relu')(avg_pool)

flatten = keras.layers.Flatten()(conv3)

output = keras.layers.Dense(N_CLASSES, activation='softmax')(flatten)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [8]:
model._name = 'combined-model-v1'

In [9]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_auc', 
                                              verbose=1,
                                              patience=7,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+".tf")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [ ]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
1037/1037 [==============================] - ETA: 0s - loss: 19.1278 - auc: 0.6842 - accuracy: 0.6125

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


1037/1037 [==============================] - 814s 738ms/step - loss: 19.1278 - auc: 0.6842 - accuracy: 0.6125 - val_loss: 1.0232 - val_auc: 0.7490 - val_accuracy: 0.6629
Epoch 2/15
1037/1037 [==============================] - ETA: 0s - loss: 12.3806 - auc: 0.8328 - accuracy: 0.7439

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


1037/1037 [==============================] - 764s 736ms/step - loss: 9.7262 - auc: 0.8818 - accuracy: 0.7959 - val_loss: 0.8552 - val_auc: 0.8148 - val_accuracy: 0.7236
Epoch 4/15
1037/1037 [==============================] - ETA: 0s - loss: 7.2618 - auc: 0.9204 - accuracy: 0.8435

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v1.tf/assets


1037/1037 [==============================] - 763s 735ms/step - loss: 7.2618 - auc: 0.9204 - accuracy: 0.8435 - val_loss: 0.6855 - val_auc: 0.8755 - val_accuracy: 0.7931
Epoch 5/15
1037/1037 [==============================] - 708s 682ms/step - loss: 5.9099 - auc: 0.9379 - accuracy: 0.8651 - val_loss: 0.9636 - val_auc: 0.8144 - val_accuracy: 0.7471
Epoch 6/15
1037/1037 [==============================] - 712s 686ms/step - loss: 4.2142 - auc: 0.9624 - accuracy: 0.9004 - val_loss: 0.8077 - val_auc: 0.8646 - val_accuracy: 0.8086
Epoch 7/15
 733/1037 [====================>.........] - ETA: 3:16 - loss: 3.2649 - auc: 0.9757 - accuracy: 0.9223